# hw2 q1

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

读取数据集

因为只要对年月日分组,把没有用的具体时刻信息去掉,把bndno缺失值赋值为单独的品牌,同时转成datetime格式

In [2]:
dataset = pd.read_csv("trade_new.csv").fillna(0)
dataset['sldatime']=pd.to_datetime(dataset['sldatime'])
dataset['sldatime']=[datetime.strftime(x,'%Y-%m-%d') for x in dataset['sldatime']]
dataset['pluno']=dataset['pluno'].astype("str")
dataset['pl1']=[x[:2] for x in dataset['pluno']]
dataset['pl2']=[x[:3] for x in dataset['pluno']]
dataset['pl3']=[x[:4] for x in dataset['pluno']]
dataset['pl4']=[x[:5] for x in dataset['pluno']]

In [3]:
bndnos = list(dataset["bndno"].astype("int").astype("str"))
for i in range(len(dataset['bndno'])):
    if bndnos[i]=='0':
        bndnos[i]="m"+str(i)
dataset["bndno"]=bndnos

In [4]:
dataset.set_index('sldatime',drop=False,inplace=True)
dataset.index=pd.DatetimeIndex(dataset.index)

In [5]:
dataset.head(10)

,uid,sldatime,pno,cno,cmrid,vipno,id,pluno,bcd,pluname,...,amt,disamt,ismmx,mtype,mdocno,isdel,pl1,pl2,pl3,pl4
sldatime,,,,,,,,,,,,,,,,,,,,,
2016-07-29,16072913541329219,2016-07-29,13,8323,男[45 以上],2900003115009,2,22002240,200328600506004228,红油桃（中）,...,5.06,0.0,0,0,0.000000e+00,0,22,220,2200,22002
2016-06-08,16060809581811553,2016-06-08,18,8334,女[18 - 25],2900001575201,5,34150006,6926458841290,MSU男童平脚裤 74129,...,27.90,0.0,0,0,0.000000e+00,0,34,341,3415,34150
2016-02-24,16022416071404989,2016-02-24,14,8305,女[31 - 45],1591040161114,2,11054040,6910019008819,雕牌超效加酶洗衣粉4000g,...,33.90,0.0,0,0,0.000000e+00,0,11,110,1105,11054
2016-05-21,16052118091609506,2016-05-21,16,8308,女[18 - 25],1591016439575,2,27400855,211211400800000011,菜煎饼,...,8.00,0.0,0,0,0.000000e+00,0,27,274,2740,27400
2016-04-03,16040315451414179,2016-04-03,14,8306,女[18 - 25],1591015027858,2,11110121,111101214,欧莱雅系列,...,39.00,0.0,0,0,0.000000e+00,0,11,111,1111,11110
2016-07-23,16072318461403789,2016-07-23,14,8305,女[18 - 25],2900000476042,12,15110032,6922577704553,君乐宝法式斯浓布丁90g,...,5.90,0.0,0,0,0.000000e+00,0,15,151,1511,15110
2016-05-15,16051519061201422,2016-05-15,12,8322,女[26 - 30],1590140305107,3,27400855,211211400800000011,菜煎饼,...,8.00,0.0,0,0,0.000000e+00,0,27,274,2740,27400
2016-07-23,16072318461403789,2016-07-23,14,8305,女[18 - 25],2900000476042,13,15110032,6922577704553,君乐宝法式斯浓布丁90g,...,5.90,0.0,0,0,0.000000e+00,0,15,151,1511,15110
2016-05-24,16052410421803977,2016-05-24,18,8302,女[18 - 25],1590142197076,6,27400855,211211400800000011,菜煎饼,...,8.00,0.0,0,0,0.000000e+00,0,27,274,2740,27400


要对pluno,dptno,bndno按年月日按销量进行汇总统计

In [6]:
def generate_time_series(period,dataset,metric,target):
    pluno_series = dict()
    dataset_time=dataset.to_period(period)
    dataset_time.sort_index(inplace=True)
    for freq in np.unique(dataset_time.index):
        orders=dataset_time.loc[freq,[metric,target]]
        pluno_series[freq]=dict()
        for index,row in orders.iterrows():
            pluno_series[freq][row[metric]]=row[target] if row[target] not in pluno_series[freq] else pluno_series[freq][row[metric]]+row[target]
    return pd.DataFrame(pluno_series).fillna(0)

1. pluno 按月、周、日统计(显示前十个)

In [7]:
#按月
generate_time_series("M",dataset,'pluno','qty').head(10)

,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07
10000000,1.0,1.0,0.0,0.0,1.0,0.0
10000003,0.0,1.0,0.0,0.0,0.0,1.0
10000004,1.0,0.0,1.0,1.0,1.0,1.0
10000005,1.0,0.0,0.0,1.0,1.0,1.0
10000006,1.0,1.0,1.0,1.0,1.0,1.0
10000007,1.0,1.0,1.0,0.0,1.0,0.0
10000009,1.0,1.0,0.0,1.0,1.0,0.0
10000010,1.0,0.0,0.0,1.0,1.0,1.0
10000011,1.0,1.0,1.0,1.0,1.0,1.0
10000012,1.0,1.0,1.0,1.0,0.0,1.0


In [8]:
#按星期
generate_time_series("W",dataset,'pluno','qty').head(10)

,2016-02-01/2016-02-07,2016-02-08/2016-02-14,2016-02-15/2016-02-21,2016-02-22/2016-02-28,2016-02-29/2016-03-06,2016-03-07/2016-03-13,2016-03-14/2016-03-20,2016-03-21/2016-03-27,2016-03-28/2016-04-03,2016-04-04/2016-04-10,...,2016-05-23/2016-05-29,2016-05-30/2016-06-05,2016-06-06/2016-06-12,2016-06-13/2016-06-19,2016-06-20/2016-06-26,2016-06-27/2016-07-03,2016-07-04/2016-07-10,2016-07-11/2016-07-17,2016-07-18/2016-07-24,2016-07-25/2016-07-31
10000000,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
10000003,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
10000004,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
10000005,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
10000006,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,...,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
10000007,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
10000009,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
10000010,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
10000011,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
10000012,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [9]:
#按日
generate_time_series("D",dataset,'pluno','qty').head(10)

,2016-02-01,2016-02-02,2016-02-03,2016-02-04,2016-02-05,2016-02-06,2016-02-07,2016-02-08,2016-02-09,2016-02-10,...,2016-07-22,2016-07-23,2016-07-24,2016-07-25,2016-07-26,2016-07-27,2016-07-28,2016-07-29,2016-07-30,2016-07-31
10000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10000003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10000004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10000005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10000006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10000007,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10000009,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10000010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
10000011,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10000012,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


2. 品类结构 按月、周、日统计(显示前十个)

In [10]:
#按月
generate_time_series("M",dataset,'pl1','qty').head(10)

,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07
10,10.000,1.000,1.000,1.000,1.000,1.000
11,1.000,1.000,1.000,1.000,1.000,1.000
14,0.434,1.000,1.000,1.000,1.000,1.000
15,1.000,1.000,1.000,1.000,2.000,1.000
20,0.514,242.000,0.010,0.003,0.124,0.800
21,0.426,0.124,0.900,1.054,0.764,0.244
22,0.946,0.396,0.666,392.000,0.760,0.634
23,1.000,1.000,1.000,10.000,10.000,1.000
24,0.466,0.304,0.378,1.534,0.100,0.990
25,0.304,0.200,2.000,0.112,0.298,0.182


In [11]:
generate_time_series("M",dataset,'pl2','qty').head(10)

,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07
100,1.0,1.0,1.0,1.0,1.0,1.0
101,1.0,1.0,1.0,1.0,1.0,1.0
102,1.0,1.0,1.0,1.0,1.0,1.0
103,2.0,1.0,1.0,1.0,1.0,1.0
104,1.0,1.0,1.0,2.0,1.0,1.0
105,10.0,1.0,1.0,10.0,1.0,0.0
106,1.0,0.0,0.0,0.0,0.0,0.0
107,0.0,0.0,0.0,18.0,1.0,0.0
110,1.0,1.0,1.0,1.0,1.0,1.0
111,1.0,1.0,1.0,1.0,1.0,1.0


In [12]:
generate_time_series("M",dataset,'pl3','qty').head(10)

,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07
1000,1.0,1.0,1.0,1.0,1.0,1.0
1010,1.0,0.0,1.0,1.0,1.0,1.0
1011,1.0,1.0,1.0,1.0,1.0,1.0
1012,0.0,0.0,0.0,0.0,1.0,0.0
1013,1.0,1.0,1.0,1.0,1.0,1.0
1014,1.0,1.0,1.0,1.0,1.0,1.0
1015,1.0,1.0,1.0,1.0,1.0,1.0
1020,1.0,1.0,1.0,1.0,1.0,1.0
1030,2.0,1.0,1.0,1.0,12.0,1.0
1031,0.0,1.0,1.0,1.0,1.0,1.0


In [13]:
generate_time_series("M",dataset,'pl4','qty').head(10)

,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07
10000,1.0,1.0,1.0,1.0,1.0,1.0
10001,1.0,1.0,1.0,1.0,1.0,1.0
10002,1.0,1.0,1.0,1.0,1.0,1.0
10008,1.0,0.0,1.0,1.0,1.0,0.0
10100,1.0,0.0,0.0,1.0,1.0,1.0
10101,1.0,0.0,0.0,0.0,0.0,0.0
10102,1.0,0.0,0.0,0.0,1.0,0.0
10103,1.0,0.0,0.0,1.0,0.0,1.0
10109,1.0,0.0,1.0,1.0,1.0,1.0
10110,1.0,1.0,1.0,1.0,1.0,1.0


In [14]:
#按周
generate_time_series("W",dataset,'pl1','qty').head(10)

,2016-02-01/2016-02-07,2016-02-08/2016-02-14,2016-02-15/2016-02-21,2016-02-22/2016-02-28,2016-02-29/2016-03-06,2016-03-07/2016-03-13,2016-03-14/2016-03-20,2016-03-21/2016-03-27,2016-03-28/2016-04-03,2016-04-04/2016-04-10,...,2016-05-23/2016-05-29,2016-05-30/2016-06-05,2016-06-06/2016-06-12,2016-06-13/2016-06-19,2016-06-20/2016-06-26,2016-06-27/2016-07-03,2016-07-04/2016-07-10,2016-07-11/2016-07-17,2016-07-18/2016-07-24,2016-07-25/2016-07-31
10,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,...,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000
11,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,...,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000
14,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,...,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,0.134,1.000
15,1.000,1.000,1.000,1.000,2.000,1.000,1.000,6.000,6.000,1.000,...,6.000,1.000,1.000,1.000,6.000,1.000,1.000,16.000,5.000,1.000
20,1.766,378.000,0.514,0.394,0.250,0.162,0.004,0.374,0.538,0.260,...,0.124,0.948,0.118,0.344,0.948,0.500,0.388,3.000,0.000,0.001
21,3.100,0.510,2.216,0.170,0.284,0.790,0.001,0.774,1.228,0.092,...,0.756,1.062,0.616,0.278,0.846,0.608,0.439,0.780,0.298,0.606
22,0.448,0.298,0.002,0.001,0.516,0.794,1.042,0.196,5.000,0.338,...,1.348,1.666,0.972,2.298,1.092,0.472,0.724,0.678,0.792,0.432
23,0.606,0.001,1.000,1.000,1.000,0.001,0.254,1.000,1.000,6.000,...,10.000,0.982,10.000,1.000,0.306,0.001,0.220,0.736,0.001,0.352
24,0.932,0.434,0.358,0.166,0.350,0.518,0.534,0.298,0.258,0.286,...,0.218,0.278,0.642,0.594,1.340,0.520,0.786,0.652,0.284,0.330
25,0.352,2.000,3.000,1.508,0.244,0.004,0.200,4.000,0.312,0.200,...,0.128,2.000,0.004,0.300,2.000,2.000,0.005,2.000,0.546,0.502


In [15]:
generate_time_series("W",dataset,'pl2','qty').head(10)

,2016-02-01/2016-02-07,2016-02-08/2016-02-14,2016-02-15/2016-02-21,2016-02-22/2016-02-28,2016-02-29/2016-03-06,2016-03-07/2016-03-13,2016-03-14/2016-03-20,2016-03-21/2016-03-27,2016-03-28/2016-04-03,2016-04-04/2016-04-10,...,2016-05-23/2016-05-29,2016-05-30/2016-06-05,2016-06-06/2016-06-12,2016-06-13/2016-06-19,2016-06-20/2016-06-26,2016-06-27/2016-07-03,2016-07-04/2016-07-10,2016-07-11/2016-07-17,2016-07-18/2016-07-24,2016-07-25/2016-07-31
100,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
101,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
102,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,6.0,...,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0
103,0.0,24.0,2.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,...,24.0,1.0,1.0,1.0,1.0,1.0,4.0,1.0,1.0,0.0
104,1.0,1.0,1.0,3.0,1.0,0.0,1.0,1.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,12.0
105,10.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,10.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
106,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
107,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,12.0,1.0,0.0,0.0,0.0,0.0,0.0
110,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
111,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,39.0,1.0,...,1.0,148.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [16]:
generate_time_series("W",dataset,'pl3','qty').head(10)

,2016-02-01/2016-02-07,2016-02-08/2016-02-14,2016-02-15/2016-02-21,2016-02-22/2016-02-28,2016-02-29/2016-03-06,2016-03-07/2016-03-13,2016-03-14/2016-03-20,2016-03-21/2016-03-27,2016-03-28/2016-04-03,2016-04-04/2016-04-10,...,2016-05-23/2016-05-29,2016-05-30/2016-06-05,2016-06-06/2016-06-12,2016-06-13/2016-06-19,2016-06-20/2016-06-26,2016-06-27/2016-07-03,2016-07-04/2016-07-10,2016-07-11/2016-07-17,2016-07-18/2016-07-24,2016-07-25/2016-07-31
1000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1010,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1011,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1012,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1013,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1014,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0
1015,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1020,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,6.0,...,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0
1030,0.0,24.0,2.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,24.0,1.0,0.0,0.0,0.0,1.0,4.0,1.0,1.0,0.0
1031,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0


In [17]:
generate_time_series("W",dataset,'pl4','qty').head(10)

,2016-02-01/2016-02-07,2016-02-08/2016-02-14,2016-02-15/2016-02-21,2016-02-22/2016-02-28,2016-02-29/2016-03-06,2016-03-07/2016-03-13,2016-03-14/2016-03-20,2016-03-21/2016-03-27,2016-03-28/2016-04-03,2016-04-04/2016-04-10,...,2016-05-23/2016-05-29,2016-05-30/2016-06-05,2016-06-06/2016-06-12,2016-06-13/2016-06-19,2016-06-20/2016-06-26,2016-06-27/2016-07-03,2016-07-04/2016-07-10,2016-07-11/2016-07-17,2016-07-18/2016-07-24,2016-07-25/2016-07-31
10000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
10001,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
10002,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
10008,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
10100,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0
10101,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10102,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
10103,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
10109,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
10110,1.0,2.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [18]:
#按日
generate_time_series("D",dataset,'pl1','qty').head(10)

,2016-02-01,2016-02-02,2016-02-03,2016-02-04,2016-02-05,2016-02-06,2016-02-07,2016-02-08,2016-02-09,2016-02-10,...,2016-07-22,2016-07-23,2016-07-24,2016-07-25,2016-07-26,2016-07-27,2016-07-28,2016-07-29,2016-07-30,2016-07-31
10,1.000,1.000,6.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,...,1.000,1.000,1.000,1.000,1.000,6.000,1.000,1.000,1.000,1.000
11,1.000,1.000,1.000,1.000,1.000,3.000,1.000,1.000,0.000,1.000,...,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000
14,1.000,1.000,1.000,1.000,0.434,1.000,1.000,1.000,1.000,1.000,...,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,0.356,1.000
15,0.310,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,6.000,...,1.000,12.000,1.000,2.000,1.000,1.000,1.000,1.000,6.000,1.000
20,1.518,0.000,0.554,0.704,0.552,0.786,0.000,0.000,0.000,434.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.045
21,0.448,0.426,0.102,0.001,0.000,0.194,0.000,0.000,0.000,0.510,...,0.122,0.298,0.428,0.000,0.000,0.270,0.000,0.000,0.694,0.606
22,0.430,1.010,0.642,1.066,0.732,0.598,1.512,0.404,0.244,0.280,...,0.552,1.282,0.900,1.302,1.088,0.001,0.738,0.432,0.966,0.900
23,1.000,0.208,1.000,10.000,0.001,10.000,1.000,0.430,10.000,0.448,...,0.432,0.001,15.000,10.000,1.000,1.000,0.001,0.284,0.001,0.156
24,0.438,1.098,0.240,0.390,0.836,0.190,0.482,0.452,0.840,0.606,...,0.384,0.576,0.600,3.156,1.114,0.412,0.000,0.990,0.448,0.396
25,0.352,0.290,2.000,0.200,0.308,2.000,0.020,0.000,0.001,0.004,...,3.000,4.000,0.546,0.000,0.001,0.200,0.200,3.000,2.000,0.005


In [19]:
generate_time_series("D",dataset,'pl2','qty').head(10)

,2016-02-01,2016-02-02,2016-02-03,2016-02-04,2016-02-05,2016-02-06,2016-02-07,2016-02-08,2016-02-09,2016-02-10,...,2016-07-22,2016-07-23,2016-07-24,2016-07-25,2016-07-26,2016-07-27,2016-07-28,2016-07-29,2016-07-30,2016-07-31
100,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,...,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
101,6.0,1.0,6.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
102,1.0,1.0,4.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
103,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
104,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,1.0,6.0,1.0,1.0,0.0,0.0
105,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
106,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
107,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
110,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
111,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [20]:
generate_time_series("D",dataset,'pl3','qty').head(10)

,2016-02-01,2016-02-02,2016-02-03,2016-02-04,2016-02-05,2016-02-06,2016-02-07,2016-02-08,2016-02-09,2016-02-10,...,2016-07-22,2016-07-23,2016-07-24,2016-07-25,2016-07-26,2016-07-27,2016-07-28,2016-07-29,2016-07-30,2016-07-31
1000,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,...,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
1010,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
1011,6.0,1.0,6.0,0.0,10.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0
1012,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1013,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0
1014,0.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,...,1.0,1.0,0.0,1.0,1.0,1.0,1.0,2.0,0.0,1.0
1015,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,...,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
1020,1.0,1.0,4.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1030,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1031,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
generate_time_series("D",dataset,'pl4','qty').head(10)

,2016-02-01,2016-02-02,2016-02-03,2016-02-04,2016-02-05,2016-02-06,2016-02-07,2016-02-08,2016-02-09,2016-02-10,...,2016-07-22,2016-07-23,2016-07-24,2016-07-25,2016-07-26,2016-07-27,2016-07-28,2016-07-29,2016-07-30,2016-07-31
10000,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0
10001,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
10008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
10101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10102,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10103,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10109,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
10110,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


3. bndno 按月、周、日统计(显示前十个)

In [22]:
#按月
generate_time_series("M",dataset,'bndno','qty').head(10)

,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07
10003,1.0,0.0,0.0,1.0,1.0,1.0
10005,0.0,0.0,0.0,0.0,1.0,0.0
10007,1.0,1.0,1.0,1.0,1.0,1.0
10010,1.0,0.0,0.0,0.0,0.0,0.0
10012,1.0,1.0,1.0,2.0,0.0,1.0
10015,1.0,1.0,0.0,0.0,4.0,1.0
10029,0.0,0.0,0.0,0.0,0.0,1.0
10033,1.0,1.0,1.0,0.0,1.0,1.0
10036,0.0,0.0,0.0,1.0,0.0,0.0
10038,1.0,0.0,0.0,1.0,0.0,0.0


In [23]:
#按周
generate_time_series("W",dataset,'bndno','qty').head(10)

,2016-02-01/2016-02-07,2016-02-08/2016-02-14,2016-02-15/2016-02-21,2016-02-22/2016-02-28,2016-02-29/2016-03-06,2016-03-07/2016-03-13,2016-03-14/2016-03-20,2016-03-21/2016-03-27,2016-03-28/2016-04-03,2016-04-04/2016-04-10,...,2016-05-23/2016-05-29,2016-05-30/2016-06-05,2016-06-06/2016-06-12,2016-06-13/2016-06-19,2016-06-20/2016-06-26,2016-06-27/2016-07-03,2016-07-04/2016-07-10,2016-07-11/2016-07-17,2016-07-18/2016-07-24,2016-07-25/2016-07-31
10003,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
10005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
10007,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
10010,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10012,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
10015,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,4.0,0.0,0.0,0.0,1.0,0.0
10029,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
10033,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,...,0.0,1.0,6.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0
10036,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10038,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
#按日
generate_time_series("D",dataset,'bndno','qty').head(10)

,2016-02-01,2016-02-02,2016-02-03,2016-02-04,2016-02-05,2016-02-06,2016-02-07,2016-02-08,2016-02-09,2016-02-10,...,2016-07-22,2016-07-23,2016-07-24,2016-07-25,2016-07-26,2016-07-27,2016-07-28,2016-07-29,2016-07-30,2016-07-31
10003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10007,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10010,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10012,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10029,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10033,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10036,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10038,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
